In [27]:
import pandas as pd
import json
import requests
import requesocks
import time
from bs4 import BeautifulSoup as BS
import re

In [135]:
cd ../metrics

/Users/michaelhood/Documents/projects/project/data/metrics


In [140]:
df = pd.read_csv('seattle_test.csv', index_col=0).reset_index()
df = df.drop('index', axis=1)

In [107]:
def string_to_dict(input_string):
    new_dict = {}
    schools_text = input_string.split('\n\n') # input_string was a giant string of school ranking info I copied off of a webpage
    for line in schools_text:
        info = line.split('\n')
        if len(info) == 2:
            score, school = info[0], info[1]
            new_dict[school] = score
        if len(info) == 3:
            score, school = info[1], info[2]
            new_dict[school] = score
        if len(info) > 3:
            score, school = info[0], info[1]
            new[school] = score
            score, school = info[3], info[4]
            new_dict[school] = score
    return new_dict

In [155]:
#clean up the few school entries that didn't quite work

d[d['']] = 1 # update Middle College High School, which is the value of d['']
d['Franklin High School'] = 7
del d['Public \xe2\x80\xa2 Grades 9-12']
del d['']

In [71]:
# write the dict to a txt file, so I can read it in later
with open('seattle_schools.txt', 'w') as f:
    json.dump(d, f)

# Now to scrap noodle to get school info

In [268]:
# create a function to compute an average school score from the html source
# for a specific address

def compute_school_average(html_source, school_dict):
    school_count = 0
    address_score = 0
    for school in school_dict.keys(): # school_dict is the school info from the string_to_dict function
        m = re.search(school, html_source)
        if m:
            school_count += 1
            address_score += int(school_dict[school])
    if school_count == 0:
        return 0
    return float(address_score) / school_count # returns an average of all the schools within ten miles

In [100]:
def test(df, start, stop):
    params = {'user-agent': 'Mozilla/5.0'}
    session = requesocks.session()
    #Use Tor for both HTTP and HTTPS
    session.proxies = {'http': 'socks5://127.0.0.1:9150',
                       'https': 'socks5://127.0.0.1:9150'}
    
    for row in df.ix[start:stop].iterrows(): # iterate through each address in the df and get the html for it
        query = (row[1][5]+' '+row[1][1]+' '+row[1][4]+' '+str(row[1][6])).replace(' ', '%20')
        # this URL is formatted to query www.noodle.com/schools
        URL = "https://www.noodle.com/search/schools?distance=10&location={0}".format(query)
        query_url = URL + query + "&sort=best_fit"
        try:
            r = session.get(query_url, headers=params)
        except:
            print "Row {0} HTTP get failed.".format(row[0])
        with open("school_index_{0}.txt".format(row[0]), 'w') as f: # write each html to a txt file so I can parse it later
            f.write(r.content)
            
        print "writing {0}".format(row[0])
    
    return
        

In [265]:
# load the file, so I can use my compute_school_average function on the html

with open('seattle_schools.txt', 'r') as f:
    d = f.read()
    school_dict = json.loads(d)

In [127]:
cd ../schools

/Users/michaelhood/Documents/projects/project/data/schools


In [263]:
#test(df, 0, len(df))

In [266]:
def aggregate_scores(df, school_dict):
    school_scores = []
    for row in df.iterrows():
        with open("school_index_{0}.txt".format(row[0]), 'r') as f:
            doc = f.read()
        school_scores.append(compute_school_average(doc, school_dict))
    return school_scores

In [151]:
cd ../schools

/Users/michaelhood/Documents/projects/project/data/schools


In [269]:
school_scores = aggregate_scores(df, school_dict)

In [170]:
df_scores = pd.DataFrame({'school_index': school_scores}, index=range(len(school_scores)))

In [172]:
df['school_index'] = df_scores

In [183]:
df['city'].unique()

array(['Burien', 'Seattle', 'Des Moines', 'Bellevue', 'Issaquah',
       'Sammamish', 'Redmond', 'Duvall', 'Kirkland'], dtype=object)

In [185]:
# impute the zeros, which are missing values, with the mean for a particular city 
for city in df['city'].unique():
    mean_score = df['school_index'][df['city'] == city].mean()
    df['school_index'][(df['city'] == city) & (df['school_index'] == 0)] = mean_score

/Users/michaelhood/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
